# 2-D Gaussian Gibbs Sampler

- Andrew J. Graves
- 10/26/20

In [1]:
# Import modules
import numpy as np
from scipy.stats import multivariate_normal

# Example parameters of 2D Gaussian
mu = np.array([2, 2])
sigma = np.array([[1, -.3], [-.3, 1]])

For the case of a 2-dimensional Gaussian random variable, it can be shown that the conditional distribution of $x_1$ given $x_2$ has the form:

$$p(x_1 | x_2 = k) = \frac{1}{\sigma\sqrt{2 \pi}}\text{exp}\left(\frac{-1}{2\sigma^2}(x_1 - \mu)^2\right)$$

It can be that the solution for the parameterization of this conditional distribution is ([source here](https://online.stat.psu.edu/stat505/lesson/6/6.1))

$$\mu_{x_1|x_2} = \mu_1 + \frac{\sigma_{12}}{\sigma_{22}}(x_2 - \mu_2)$$

$$\sigma_{x_1|x_2} = \sigma_{11} - \frac{\sigma_{12}^2}{\sigma_{22}}$$

We can use these updating equations to make random draws from univariate slices of the bivariate Gaussian. This simple setup with a Gibbs sampler will converge to the target distribution relatively quickly.

In [2]:
# Convenience functions for updating parameters on each draw
def x1_giv_x2(x2, mu, sigma):
    # Update x1 given x2
    new_mu = mu[0] + sigma[0, 1] / sigma[1, 1] * (x2 - mu[1])
    new_sigma = sigma[0, 0] - (sigma[0, 1] ** 2) / sigma[1, 1]
    x1 = np.random.normal(new_mu, new_sigma)
    return x1

def x2_giv_x1(x1, mu, sigma):
    # Update x2 given x1
    new_mu = mu[1] + sigma[1, 0] / sigma[0, 0] * (x1 - mu[0])
    new_sigma = sigma[1, 1] - (sigma[1, 0] ** 2) / sigma[0, 0]
    x2 = np.random.normal(new_mu, new_sigma)
    return x2

# Gibbs sampler
def gibbs_sampler_gauss(mu, sigma, draw=100, tune=100, init=np.random.rand()):
    
    # Initialize sampling output
    samp = np.zeros((draw, 2))
    
    # Initial value for x2
    x2 = init

    # Tune the sampler
    for i in range(tune):
        # Update locations and parameters
        x1 = x1_giv_x2(x2, mu, sigma)
        x2 = x2_giv_x1(x1, mu, sigma)
    
    # Draw from the 2D distribution
    for i in range(draw):
        # Update locations and parameters
        x1 = x1_giv_x2(x2, mu, sigma)
        x2 = x2_giv_x1(x1, mu, sigma)
        samp[i, :] = [x1, x2]
        
    return samp

# Evaluate the 2D Gaussian on inputs
rv = multivariate_normal(mu, sigma)